# Projeto de detecção de íris

In [1]:
# Imports
import cv2
import os
import G6_iris_recognition as g6
import matplotlib.pyplot as plt
import numpy as np
from random import randint
from time import time

## Tratando as imagens

### Melhorando a nitidez

In [2]:
# Funcao que retorna uma imagem mais nitida usando uma mascara de nitidez
# Obtida em https://stackoverflow.com/a/55590133/9771897
def sharpen_image(img, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """
    img = imagem de entrada
    kernel_size = tamanho do kernel
    sigma = valor de sigma a ser usado no GaussianBlur
    amount = intensidade da nitidez: quanto maior, mais nitida a imagem
    threshold = limite a ser aplicado na mascara de baixo contraste
    """
    blurred = cv2.GaussianBlur(img, kernel_size, sigma)
    sharpened = float(amount + 1) * img - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    
    if threshold > 0:
        low_contrast_mask = np.absolute(img - blurred) < threshold
        np.copyto(sharpened, img, where=low_contrast_mask)
        
    return sharpened

### Melhorando o brilho e o contraste

De acordo com a [documentação oficial](https://docs.opencv.org/master/d3/dc1/tutorial_basic_linear_transform.html) do OpenCV, a melhor forma de fazer ajustes no brilho e no contraste é utilizando a fórmula abaixo

$img_{out} = \alpha \cdot img_{in} + \beta$

onde: 
- $img_{out}$ é a imagem após ser processada;
- $\alpha$ é o parâmetro de _gain_ (utilizado para controlar o contraste);
- $img_{in}$ é a imagem a ser processada;
- $\beta$ é o parâmetro de _bias_ (utilizado para controlar o brilho);

Para ajustes no contraste, basta alterar o valor de $\alpha$:
- para $\alpha = 1$, a imagem não tem alteração de contraste;
- para $0 < \alpha < 1$, o contraste da imagem é reduzido;
- para $\alpha > 1$, o contraste da imagem é intensificado;

Para ajustes no brilho, basta alterar o valor de $\beta$. O range ideal de brilho é tal que $-127 < \beta < 127$, mas podem ser utilizados outros valores.

De modo a fazer esta conta, podemos usar a função [`addWeighted`](https://docs.opencv.org/master/d2/de8/group__core__array.html#gafafb2513349db3bcff51f54ee5592a19) do cv2, que faz a soma de duas matrizes utilizando pesos de acordo com a fórmula abaixo.

$out = M_1 \cdot \alpha + M_2 \cdot \beta + \gamma$

Como estamos tratando apenas uma imagem, basta utilizar o valor de $beta$ como 0, gerando uma fórmula idêntica à de manipulação de contraste e de brilho.

In [3]:
# Funcao que ajusta simultaneamente o brilho e o contraste
def adjust_brightness_and_contrast(img, alpha=1, beta=0):
    """
    img = imagem de entrada
    alpha = valor de contraste
    """
    # Se o ajuste de contraste for 1 e o ajuste de brilho for 0, a imagem sera inalterada
    if alpha == 1 and beta == 0:
        return img
    else:
        if alpha < 0:
            raise ValueError("alpha value can't be lower than 0!")
        
        else:
            return cv2.addWeighted(img, alpha, img, 0, beta)

### Tratando e armazenando as novas imagens

Foi feito um teste utilizando a técnica de equalização do histograma de cores da imagem; no entanto, este gerou imagens com muito brilho nas pupilas e pouco destaque para as íris. Assim, realizou-se apenas ajustes na nitidez, no contraste e no brilho das imagens.

Com equalização            |  Sem equalização
:-------------------------:|:-------------------------:
![](with_eq.bmp)           |![](without_eq.bmp)

In [4]:
# Gerando pasta para salvar as imagens tratadas
if not os.path.isdir("enhanced_imgs"):
    os.mkdir("enhanced_imgs")

In [5]:
# Gerando lista com os numeros das pastas
img_list = [f"{str(i).zfill(4)}" for i in range(60) if i != 12] # Nao existe pasta 0012

# A maior parte das pastas contém 20 imagens cada
n_imgs = 20

In [6]:
# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    # Checa se a pasta nao existe, e cria ela se nao existir
    if not os.path.isdir(enhanced_path):
        os.mkdir(os.path.join("enhanced_imgs", path))
        
    # Percorrendo cada imagem dentro da pasta
    for i in range(n_imgs):
        # Obtem o path completo da imagem
        filename = f"{path}_{str(i).zfill(3)}.bmp"
        img_path = os.path.join("images", path, filename)        
        
        # Existem casos em que uma pasta nao possui 20 imagens
        # Verifica-se portanto se a imagem existe antes de processar
        if not os.path.isfile(img_path):
            pass
        
        else:
            img = cv2.imread(img_path)
            
            # Tratando
            sharpened = sharpen_image(img, amount=2.0)
            processed = adjust_brightness_and_contrast(sharpened, alpha = 1.5, beta = 40)

            # Salvando a imagem
            path_processed = os.path.join("enhanced_imgs", path, filename)
            cv2.imwrite(path_processed, processed)

## Treinando o modelo

In [7]:
g6.iris_model_train("enhanced_imgs", "model.pickle")

directory_list ['enhanced_imgs/0027', 'enhanced_imgs/0032', 'enhanced_imgs/0023', 'enhanced_imgs/0001', 'enhanced_imgs/0022', 'enhanced_imgs/0037', 'enhanced_imgs/0025', 'enhanced_imgs/0004', 'enhanced_imgs/0031', 'enhanced_imgs/0034', 'enhanced_imgs/0010', 'enhanced_imgs/0007', 'enhanced_imgs/0020', 'enhanced_imgs/0054', 'enhanced_imgs/0005', 'enhanced_imgs/0000', 'enhanced_imgs/0018', 'enhanced_imgs/0029', 'enhanced_imgs/0014', 'enhanced_imgs/0006', 'enhanced_imgs/0046', 'enhanced_imgs/0051', 'enhanced_imgs/0050', 'enhanced_imgs/0058', 'enhanced_imgs/0009', 'enhanced_imgs/0008', 'enhanced_imgs/0042', 'enhanced_imgs/0026', 'enhanced_imgs/0003', 'enhanced_imgs/0053', 'enhanced_imgs/0002', 'enhanced_imgs/0019', 'enhanced_imgs/0015', 'enhanced_imgs/0016', 'enhanced_imgs/0056', 'enhanced_imgs/0028', 'enhanced_imgs/0013', 'enhanced_imgs/0059', 'enhanced_imgs/0049', 'enhanced_imgs/0035', 'enhanced_imgs/0055', 'enhanced_imgs/0011', 'enhanced_imgs/0038', 'enhanced_imgs/0043', 'enhanced_imgs/0

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
invalid_image 0032
name 0023
path_to_image enhanced_imgs/0023/0023_002.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0023/0023_012.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0023/0023_001.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0023/0023_008.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0023/0023_019.bmp
locate expr

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0022/0022_019.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0022/0022_001.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0022/0022_008.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0022/0022_005.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0022/0022_015.bmp
locate expression 1 local variable 'east

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0025/0025_003.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0025/0025_009.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0025/0025_014.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0025/0025_001.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0025/0025_002.bmp
locate expression 1 local variable 'east

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0031/0031_010.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
invalid_image 0031
name 0034
path_to_image enhanced_imgs/0034/0034_003.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0034/0034_006.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0034/0034_019.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0034/0034_017.bmp
locate expr

...

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0057/0057_013.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0057/0057_001.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0057/0057_007.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0057/0057_006.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0057/0057_010.bmp
locate expression 1 local variable 'east

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0039/0039_004.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0039/0039_007.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0039/0039_017.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0039/0039_019.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
invalid_image 0039
name 0036
path_to_image enhanced_imgs/0036/0036_013.bmp
locate expr

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0024/0024_014.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
invalid_image 0024
name 0048
path_to_image enhanced_imgs/0048/0048_016.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0048/0048_019.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0048/0048_000.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0048/0048_013.bmp
locate expr

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0052/0052_003.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0052/0052_011.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0052/0052_017.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0052/0052_019.bmp
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
path_to_image enhanced_imgs/0052/0052_006.bmp
locate expression 1 local variable 'east

[]

## Testando o modelo

In [8]:
# Contadores
total_imgs = 0
matches = 0
failed = 0

# Matriz de confusao
confusion_matrix = []

# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    for image in os.listdir(enhanced_path):
        # Contando o numero de imagens total
        total_imgs += 1
        
        # Pegando o path completo do arquivo
        img_path = os.path.join(enhanced_path, image)
        
        # Passando a imagem para o modulo G6
        guess = g6.iris_model_test("model.pickle", img_path)
        temp = guess
        
        # Se nao deu match, o valor na matriz de confusao deve ser -1
        if guess == "unmatch":
            failed += 1
            temp = -1
        
        # Se der match, o guess tem que bater com o numero da pasta
        elif int(guess) == int(path):
            matches += 1
        
        confusion_matrix.append(np.array([int(path), temp]))
        
confusion_matrix = np.array(confusion_matrix)

print(f"Matches: {matches}")
print(f"Failures: {failed}")
print(f"Success rate: {round(matches/total_imgs*100, 2)}%")

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

...

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac

locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'east_mark' referenced before assignment
rectangle expression2 not enough values to unpac